# Wildmeshing tutorial

This is a jupyter notebook. The "real" notebook can be found [here](https://github.com/wildmeshing/wildmeshing-python/blob/master/examples/tutorial.ipynb).

To use it just import it!

In [ ]:
import wildmeshing as wm

We also import meshplot to visualize

In [ ]:
import meshplot as mp

## Triangulation

The 2D meshing supports high order


In [ ]:
V, F, nodes, F_nodes = wm.triangulate_svg("rocket.svg")

Plot the resulting mesh

In [ ]:
mp.plot(V, F, shading={"wireframe": True})

The resulting mesh with the high-order nodes:

In [ ]:
p = mp.plot(V, F, return_plot=True, shading={"wireframe": True})
p.add_points(nodes, shading={"point_size": 10})
p

The previous mesh also outside triangles, to avoid this we enable the option `cut_outside`

In [ ]:
V, F, nodes, F_nodes = wm.triangulate_svg("rocket.svg", cut_outside=True)

In [ ]:
p = mp.plot(V, F, return_plot=True, shading={"wireframe": True})
p.add_points(nodes, shading={"point_size": 10})
p

I we want to carve out the window we need to speficy a point inside

In [ ]:
V, F, nodes, F_nodes = wm.triangulate_svg("rocket.svg", cut_outside=True, hole_pts=[[255, 165]])

In [ ]:
import numpy as np

p = mp.plot(V, F, return_plot=True, shading={"wireframe": True})
p.add_points(nodes, shading={"point_size": 10})
p.add_points(np.array([255, 165]), shading={"point_size": 30, "point_color": "green"})
p

## Tetrahedralization (alpha)

The tetrahedralization part is still under developement, for the moment it supports only files as input and output. 

In [ ]:
wm.tetrahedralize("small.stl", "out.msh")